In [1]:
import pandas as pd
import json
import re
from string import punctuation
from konlpy.tag import Mecab

mecab = Mecab()

# pororo 라벨링, 1차 단어사전 만들기

In [ ]:
%%writefile naver_pickle.py
import pandas as pd
import json
from pororo import Pororo
import os
from tqdm import tqdm

s_nm = Pororo(task = 'sentiment',model = 'brainbert.base.ko.nsmc', lang = 'ko')
s_sh = Pororo(task = 'sentiment', model = 'brainbert.base.ko.shopping', lang = 'ko')

for f in os.listdir('Naver종토방')[1:]:
  NAVER = pd.DataFrame(columns = ['_id', 'code', 'date', 'title', 'views', 'pos', 'neg', 'nsmc', 'shop'])
  if f.endswith('json'):
    print(f)
    for line in tqdm(open('Naver종토방/'+ f, 'r', encoding = 'utf8').readlines()):
      j = json.loads(line)
      nm_sa = s_nm(j['title'], show_probs = True)['positive']
      sh_sa = s_sh(j['title'], show_probs = True)['positive']
      j.update(nsmc = nm_sa, shop = sh_sa),
      NAVER = NAVER.append(j, ignore_index = True)
      NAVER.to_pickle(f"./naver_all_{f}.pkl")

  # pd.read_pickle('./naver_all.pkl').append(pd.read_pickle('./naver_all2.pkl'), ignore_index= True).to_pickle('./naver_all.pkl')

In [ ]:
%%writefile dc_pickle.py
import pandas as pd
import json
from pororo import Pororo
import os
from tqdm import tqdm

s_nm = Pororo(task = 'sentiment',model = 'brainbert.base.ko.nsmc', lang = 'ko')
s_sh = Pororo(task = 'sentiment', model = 'brainbert.base.ko.shopping', lang = 'ko')

for f in os.listdir('DC갤러리'):
  DC = pd.DataFrame(columns = ['_id', 'code', 'date', 'title', 'views', 'pos', 'neg', 'nsmc', 'shop'])
  if f.endswith('json'):
    print(f)
    for line in tqdm(open('DC갤러리/'+ f, 'r', encoding = 'utf8').readlines()):
      j = json.loads(line)
      nm_sa = s_nm(j['title'], show_probs = True)['positive']
      sh_sa = s_sh(j['title'], show_probs = True)['positive']
      j.update(nsmc = nm_sa, shop = sh_sa)
      DC = DC.append(j, ignore_index = True)
      DC.to_pickle(f"./dc_all_{f}.pkl")

In [ ]:
# 백그라운드로 파일 실행
# !nohup /usr/bin/python3 naver_pickle.py > naver.log 2>&1 &
# !nohup /usr/bin/python3 dc_pickle.py > dc.log 2>&1 &

# log 확인
# !tail -f naver.log
# !tail -f dc.log

# 감성사전 만들기

mecab 체ㄴ, 용ㅓ, 감탄ㅏ(확ㅣ), (관형ㅏ)


{토큰 : 감ㅓ확률, 토ㄴ: 감ㅓ확률}


In [39]:
mecab.tagset

{'EC': '연결 어미',
 'EF': '종결 어미',
 'EP': '선어말어미',
 'ETM': '관형형 전성 어미',
 'ETN': '명사형 전성 어미',
 'IC': '감탄사',
 'JC': '접속 조사',
 'JKB': '부사격 조사',
 'JKC': '보격 조사',
 'JKG': '관형격 조사',
 'JKO': '목적격 조사',
 'JKQ': '인용격 조사',
 'JKS': '주격 조사',
 'JKV': '호격 조사',
 'JX': '보조사',
 'MAG': '일반 부사',
 'MAJ': '접속 부사',
 'MM': '관형사',
 'NNB': '의존 명사',
 'NNBC': '단위를 나타내는 명사',
 'NNG': '일반 명사',
 'NNP': '고유 명사',
 'NP': '대명사',
 'NR': '수사',
 'SC': '구분자 , · / :',
 'SE': '줄임표 …',
 'SF': '마침표, 물음표, 느낌표',
 'SH': '한자',
 'SL': '외국어',
 'SN': '숫자',
 'SSC': '닫는 괄호 ), ]',
 'SSO': '여는 괄호 (, [',
 'SY': '기타 기호',
 'VA': '형용사',
 'VCN': '부정 지정사',
 'VCP': '긍정 지정사',
 'VV': '동사',
 'VX': '보조 용언',
 'XPN': '체언 접두사',
 'XR': '어근',
 'XSA': '형용사 파생 접미사',
 'XSN': '명사파생 접미사',
 'XSV': '동사 파생 접미사'}

In [2]:
def data_preprocessing(pkl : str):
  df = pd.read_pickle(pkl)
  types = {i : 'int16' for i in df.columns if i not in ['_id', 'code', 'date', 'title', 'nsmc', 'shop']}
  df = df.astype(dtype = types)
  df.date = pd.to_datetime(df.date)
  # 중복 제거
  df.drop_duplicates(subset = ['code', 'title'], inplace=True, ignore_index= True)
  df['positive'] = (df.nsmc + df.shop) / 2
  # 전처리
  # Stopwords 처리
  pattern1 = re.compile(r'[{}]'.format(re.escape(punctuation)))  # punctuation 제거
  pattern2 = re.compile(r'[^가-힣! ]')  # 특수문자, 자음, 모음, 숫자, 영어 제거
  pattern3 = re.compile(r'\s{2,}')  # white space 1개로 바꾸기.

  df.title = df.title.map(lambda x: pattern3.sub(' ',
                                                    pattern2.sub('',
                                                                    pattern1.sub('', x))))
  global mecab
  mecab = Mecab()
  # mecab 토큰화
  # df['mecab'] = df.title.apply(lambda x: mecab.morphs(x))
  df['mecab'] = df.title.apply(lambda x: mecab.pos(x))
  # N(체언), V(용언), IC(감탄사)만 거르기
  for m in df.mecab.values:
    for w, t in m:
        # print(m)
        if t.startswith('N') or t.startswith("V") or t == 'IC': pass
        else: m.remove((w, t))
  
  return df

In [3]:
NAVER = data_preprocessing('./naver_all.pkl').rename(columns = {'views':'view'})
DC = data_preprocessing('./dc_all.pkl')

In [4]:
# naver = NAVER.drop(columns = ['_id', 'nsmc', 'shop'])
DATA = NAVER[['code', 'date', 'mecab', 'view', 'positive']].append(DC[['code', 'date', 'mecab', 'view', 'positive']],
                                                                    ignore_index = True)
DATA

,code,date,mecab,view,positive
0,008770,2021-10-02 18:54:00,[],5,0.030165
1,008770,2021-10-02 14:44:00,"[(애, NNG), (호텔, NNG), (주, NNG), (아니, VCN)]",116,0.057826
2,008770,2021-10-02 13:04:00,"[(솔, NNP), (브레인, NNP), (화요일, NNG), (시초, NNG), ...",204,0.856793
3,008770,2021-10-02 12:04:00,"[(기대, NNG), (크, VA), (실망, NNG), (크, VA)]",272,0.011657
4,008770,2021-10-02 10:20:00,"[(기, NNG), (막히, VV), (쌍, NNG), (바닥, NNG), (이, ...",209,0.022041
...,...,...,...,...,...
214170,041140,2022-01-03 10:33:37,"[(저번, NNG), (주, NNG), (넥슨, NNP), (지티, NNP), (햇...",76,0.242665
214171,041140,2022-01-03 10:13:53,"[(넥슨, NNP), (지티, NNP), (감사, NNG)]",66,0.502039
214172,041140,2022-01-03 10:13:36,"[(넥슨, NNP), (지티, NNP), (물린, VV+ETM), (거, NNB),...",169,0.074371
214173,041140,2022-01-02 14:19:26,"[(넷, IC), (게임즈, NNP), (넥슨, NNP), (지티, NNP), (중...",123,0.563175


In [5]:
SENTIMENT = {}

In [6]:
for data in DATA[['mecab', 'view', 'positive']].values:
    # print(data[0])
    for w, t in data[0]: # mecab tokens
        if w in SENTIMENT.keys():
            SENTIMENT[w].append((data[1], data[2]))
        else:
            SENTIMENT[w] = [(data[1], data[2])]
    # break

In [ ]:
SENTIMENT

In [7]:
for key, values in SENTIMENT.items():
    one_view = 0
    one_positive = 0
    for v in values: one_view += v[0]
    for v in values: one_positive += (v[0] / one_view) * v[1]

    SENTIMENT[key] = one_positive

In [8]:
SENTIMENT

{'애': 0.2619855903828141,
 '호텔': 0.33648721626662653,
 '주': 0.31349433799257403,
 '아니': 0.1549262767113355,
 '솔': 0.29290773345916243,
 '브레인': 0.5106700248508675,
 '화요일': 0.38252434855445994,
 '시초': 0.28714582657185306,
 '담': 0.4635188054535649,
 '기대': 0.5766325694173337,
 '크': 0.36534416197242214,
 '실망': 0.11986722270721918,
 '기': 0.23406803780607283,
 '막히': 0.15972132486032575,
 '쌍': 0.2882629699853861,
 '바닥': 0.18857798842939907,
 '이': 0.25609723777195975,
 '호텔신라': 0.33804825625916335,
 '증권사': 0.21505231342767683,
 '리포트': 0.3346187766152506,
 '게시판': 0.22297499480936714,
 '보': 0.2702903250063639,
 '난': 0.298364401597722,
 '누구': 0.16245354047552057,
 '냐면': 0.19274093258048594,
 '호신': 0.2781476924853956,
 '고점': 0.32327402282404133,
 '신호': 0.41327578692501876,
 '알려준': 0.18563435812425327,
 '초고': 0.6713415821005586,
 '수다': 0.4352911112547132,
 '행': 0.3435297469301399,
 '들': 0.3270776100835455,
 '코로나': 0.2896928330584513,
 '알약': 0.4871364096306301,
 '치료제': 0.4232539175340683,
 '폭': 0.3140

In [104]:
json.dump(SENTIMENT, open('./SENTIMENT_ver1.0.json', 'w'), ensure_ascii=False)

# 주가 등락 적용 업데이트
- 10:03 게시글 -> 긍정확률에 수식 적용하려면 % 올랐는지가 있어야 하지 -> 10시 5분에 비해 10:10분 기반으로 올랐는지 내렸는지 학습
- 18:50 게시글 -> 9:00에 비해 9:05가 올랐는지, 09:05 기반으로 학습
- pdf -> 하루종일 영향을 준다. (안 올라오는 경우가 더 많음. 올라올 때만 추가적인 정보로) +a


### 등락 수식 (감성사전 업데이트 식)
- n% 변화 (최대 30%), $x_t = t시점의 단어 감성점수$
- known에 대해서 점수 업데이트 > 문장의 감성점수를 구할 수 있음(평균) > unknown initialize
    - known 부정/긍정 :  $x_{t+1} = x_t + \frac{n}{30} * x_t * max(해당분위 감성점수)$ > $min(max(x_{t+1}, 0), 1)$ > 짜잔

ex. 삼성에 대해 100개 리뷰(9시 0분 ~ 9시 4분)  
-> 100개 리뷰 전체 word counting > 30% 30% 40% 나누고 > 해당 분위 감성점수를 더 곱해줌(위 수식)  
-> a 50번 b 2번의 경우

---------
### 종목별 확률 (inference 식) ; 예측할 때
- 가중치 : 